### Read Config File

In [1]:
import configparser

config = configparser.ConfigParser()
config.read('dwh.cfg')

KEY = config.get('AWS', 'KEY')
SECRET = config.get('AWS', 'SECRET')

ARN = config.get('IAM_ROLE', 'ARN')
LOG_DATA = config.get('S3', 'LOG_DATA')
LOG_JSONPATH = config.get('S3', 'LOG_JSONPATH')
SONG_DATA = config.get('S3', 'SONG_DATA')

NoSectionError: No section: 'AWS'

### Access the data in s3

In [3]:
import boto3

s3 = boto3.resource('s3',
                    region_name = 'us-west-2',
                    aws_access_key_id = KEY,
                    aws_secret_access_key = SECRET
                   )

a = s3.Bucket("udacity-dend")
for i in a.objects.filter(Prefix="log_json"):
    print(i.key)

log_json_path.json


### Access Redshift

In [2]:
import psycopg2

config = configparser.ConfigParser()
config.read('dwh.cfg')

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

In [18]:
# DROP TABLES
staging_events_table_drop = "DROP TABLE IF EXISTS staging_events"
staging_songs_table_drop = "DROP TABLE IF EXISTS staging_songs"
songplay_table_drop = "DROP TABLE IF EXISTS songplays"
user_table_drop = "DROP TABLE IF EXISTS users"
song_table_drop = "DROP TABLE IF EXISTS songs"
artist_table_drop = "DROP TABLE IF EXISTS artists"
time_table_drop = "DROP TABLE IF EXISTS time"

In [19]:
create_table_queries = [staging_events_table_drop, staging_songs_table_drop, songplay_table_drop, user_table_drop, song_table_drop, artist_table_drop, time_table_drop]

In [20]:
for query in create_table_queries:
    cur.execute(query)
    conn.commit()

In [21]:
# CREATE TABLES

staging_events_table_create= ("""
    CREATE TABLE IF NOT EXISTS stage_events
    (
        artist VARCHAR,
        auth VARCHAR,
        firstName VARCHAR,
        gender VARCHAR,
        itemInSession INT,
        lastName VARCHAR,
        length NUMERIC,
        level VARCHAR,
        location VARCHAR,
        method VARCHAR,
        page VARCHAR,
        registration BIGINT,
        sessionId INT,
        song VARCHAR,
        status INT,
        ts BIGINT,
        userAgent VARCHAR,
        userId INT
    );
""")

staging_songs_table_create = ("""
    CREATE TABLE IF NOT EXISTS staging_songs
    (
        num_songs INT,
        artist_id VARCHAR,
        artist_latitude NUMERIC,
        artist_longitude NUMERIC, 
        artist_location VARCHAR,
        artist_name VARCHAR,
        song_id VARCHAR, 
        title VARCHAR, 
        duration NUMERIC,
        year INT
    );
""")

songplay_table_create = ("""
    CREATE TABLE IF NOT EXISTS songplays
    (
        songplay_id INT IDENTITY(0,1) PRIMARY KEY, 
        start_time TIMESTAMP NOT NULL, 
        user_id INT NOT NULL, 
        level VARCHAR, 
        song_id VARCHAR, 
        artist_id VARCHAR, 
        session_id INT, 
        location VARCHAR, 
        user_agent VARCHAR,
        UNIQUE (start_time, user_id, song_id, artist_id)
    );
""")

user_table_create = ("""
    CREATE TABLE IF NOT EXISTS users
    (
        user_id INT PRIMARY KEY, 
        first_name VARCHAR NOT NULL, 
        last_name VARCHAR NOT NULL, 
        gender VARCHAR, 
        level VARCHAR NOT NULL
    );
""")

song_table_create = ("""
    CREATE TABLE IF NOT EXISTS songs
    (
        song_id VARCHAR PRIMARY KEY, 
        title VARCHAR NOT NULL, 
        artist_id VARCHAR, 
        year INT, 
        duration NUMERIC
    );
""")

artist_table_create = ("""
    CREATE TABLE IF NOT EXISTS artists
    (
        artist_id VARCHAR PRIMARY KEY,
        name VARCHAR NOT NULL,
        location VARCHAR,
        latitude NUMERIC,
        longitude NUMERIC
    );
""")

time_table_create = ("""
    CREATE TABLE IF NOT EXISTS time
    (
        start_time TIMESTAMP PRIMARY KEY,
        hour INT,
        day INT,
        week INT,
        month INT,
        year INT,
        weekday INT
    );
""")

In [22]:
create_table_queries = [staging_events_table_create, staging_songs_table_create, songplay_table_create, user_table_create, song_table_create, artist_table_create, time_table_create]

In [23]:
for query in create_table_queries:
    cur.execute(query)
    conn.commit()

In [37]:
# STAGING TABLES
staging_events_copy = ("""
    COPY stage_events FROM {}
    credentials 'aws_iam_role={}'
    region 'us-west-2'
    format as json {};
""").format(LOG_DATA, ARN, LOG_JSONPATH)

staging_songs_copy = ("""
    COPY staging_songs FROM {}
    credentials 'aws_iam_role={}'
    region 'us-west-2'
    json 'auto'
""").format(SONG_DATA, ARN)

In [38]:
copy_table_queries = [staging_events_copy, staging_songs_copy]

In [39]:
for query in copy_table_queries:
    cur.execute(query)
    conn.commit()

In [10]:
songplay_table_insert = ("""
    INSERT INTO songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
    SELECT 
        DISTINCT (TIMESTAMP 'epoch' + se.ts/1000 * INTERVAL '1 second') AS start_time, 
        se.userId       AS user_id,
        se.level        AS level,
        ss.num_songs    AS song_id,
        ss.artist_id    AS artist_id,
        se.sessionId    AS session_id, 
        se.location     AS location, 
        se.userAgent    AS user_agent
    FROM stage_events se
    JOIN staging_songs ss ON se.song = ss.title AND se.artist = ss.artist_name AND se.length = ss.duration
    WHERE se.page = 'NextSong' ; 
""")

user_table_insert = ("""
    INSERT INTO users (user_id, first_name, last_name, gender, level)
    SELECT
        DISTINCT se.userId      AS user_id,
        se.firstName            AS first_name, 
        se.lastName             AS last_name,
        se.gender               AS gender,
        se.level                AS level
    FROM stage_events se
    WHERE se.page = 'NextSong';
""")

song_table_insert = ("""
    INSERT INTO songs (song_id, title, artist_id, year, duration)
    SELECT
        DISTINCT ss.num_songs        AS song_id,
        ss.title                     AS title,    
        ss.artist_id                 AS artist_id,
        ss.year                      AS year,
        ss.duration                  AS duration
    FROM staging_songs ss
    WHERE song_id NOT IN (SELECT DISTINCT song_id FROM songs);
""")

artist_table_insert = ("""
    INSERT INTO artists (artist_id, name, location, latitude, longitude)
    SELECT
        DISTINCT ss.artist_id   AS artist_id,
        ss.artist_name          AS name,
        ss.artist_location      AS location,    
        ss.artist_latitude      AS latitude,
        ss.artist_longitude     AS longitude
    FROM staging_songs ss
    WHERE artist_id NOT IN (SELECT DISTINCT artist_id FROM artists);
""")

time_table_insert = ("""
    INSERT INTO time (start_time, hour, day, week, month, year, weekday)
    SELECT
        DISTINCT songplays.start_time       AS start_time,
        EXTRACT(HOUR FROM start_time)       AS hour,
        EXTRACT(DAY FROM start_time)        AS day,
        EXTRACT(WEEK FROM start_time)       AS week,
        EXTRACT(MONTH FROM start_time)      AS month,
        EXTRACT(YEAR FROM start_time)       AS year,
        EXTRACT(weekday FROM start_time)    AS weekday
    FROM songplays;
""")

In [4]:
insert_table = [songplay_table_insert, user_table_insert, song_table_insert, artist_table_insert, time_table_insert]

In [11]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

cur.execute(songplay_table_insert)
conn.commit()

In [6]:
query = """
    SELECT * FROM songplays
    LIMIT 10;
"""

In [7]:
cur.execute(query)
conn.commit()

In [8]:
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(964, datetime.datetime(2018, 11, 11, 19, 56, 23), 80, 'paid', '1', 'AR03P141187B9B8CE6', 435, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
(456, datetime.datetime(2018, 11, 15, 6, 17, 59), 80, 'paid', '1', 'AR049S81187B9AE8A5', 602, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
(472, datetime.datetime(2018, 11, 6, 16, 32, 54), 30, 'paid', '1', 'AR049S81187B9AE8A5', 130, 'San Jose-Sunnyvale-Santa Clara, CA', 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0')
(54, datetime.datetime(2018, 11, 26, 13, 21, 9), 88, 'paid', '1', 'AR0DYTO1187FB4B6AE', 900, 'Sacramento--Roseville--Arden-Arcade, CA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
(62, datetime.datetime(2018